In [ ]:
# dont_test
# default_exp scripts
%load_ext nb_black

<IPython.core.display.Javascript object>

# lib_name Workaround

For nbdev lib_name and lib_path have to be identical, but here they are not. This module implements a little workaround.

In [ ]:
# export

from pathlib import Path
from configparser import ConfigParser


def patch_config(settings_path, lib_name):
    config = ConfigParser()
    config.read(settings_path)
    config["DEFAULT"]["lib_name"] = lib_name
    with settings_path.open("w") as settings_file:
        config.write(settings_file)
    return config

<IPython.core.display.Javascript object>

### Test Helper Functions

In [ ]:
def setup_settings_file(settings_path):
    original_lib_name = "django_fileresponse"
    settings_content = f"""
    [DEFAULT]
    lib_name = {original_lib_name}
    """
    with settings_path.open("w") as f:
        f.write(settings_content)
    return settings_path, original_lib_name


def get_actual_lib_name(settings_path):
    config = ConfigParser()
    config.read(settings_path)
    return config["DEFAULT"]["lib_name"]

<IPython.core.display.Javascript object>

### Test for patch_config

In [ ]:
# arange
settings_path, original_lib_name = setup_settings_file(Path.cwd() / "test.ini")

# act
patched_lib_name = "fileresponse"
patch_config(settings_path, patched_lib_name)

# assert
actual_lib_name = get_actual_lib_name(settings_path)

assert patched_lib_name == actual_lib_name

<IPython.core.display.Javascript object>

## patch_lib_name contextmanager

In [ ]:
# export

from contextlib import contextmanager


@contextmanager
def patch_lib_name(settings_path, lib_name):
    config = ConfigParser()
    config.read(settings_path)
    original_lib_name = config["DEFAULT"]["lib_name"]
    config = patch_config(settings_path, lib_name)
    try:
        yield config
    finally:
        patch_config(settings_path, original_lib_name)

<IPython.core.display.Javascript object>

### Test for patch_lib_name contextmanager

In [ ]:
# arange
settings_path, original_lib_name = setup_settings_file(Path.cwd() / "test.ini")

# act and assert
patched_lib_name = "fileresponse"
with patch_lib_name(settings_path, patched_lib_name) as patched_config:
    actual_lib_name = get_actual_lib_name(settings_path)
    assert patched_lib_name == actual_lib_name

# assert
actual_lib_name = get_actual_lib_name(settings_path)
assert original_lib_name == actual_lib_name

<IPython.core.display.Javascript object>

In [ ]:
# export

import re
import sys

from pathlib import Path
from contextlib import contextmanager

from nbdev.export2html import nbdev_build_docs


def nbdev_build_docs_patched():
    print("hello world")
    cwd = Path.cwd()
    settings_path = cwd / "settings.ini"
    with patch_lib_name(settings_path, "fileresponse"):
        sys.argv[0] = re.sub(r"(-script\.pyw|\.exe)?$", "", sys.argv[0])
        sys.exit(nbdev_build_docs())

<IPython.core.display.Javascript object>